In [27]:
#set parameters
dataset_name = 'default' #'adult', 'bank', 'default', 'law'
split = 'attr' #'attr'
fairness_notion = 'ate' #'eqop'
num_clients = 3 #5, 10, 15


In [28]:
import seaborn as sns
%matplotlib inline

sns.set_style('darkgrid')

from sklearn.preprocessing import OneHotEncoder,MinMaxScaler,StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV,SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

import numpy as np
import random
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Input
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import mnist
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import argparse
# Importing necssary modules
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot
from sklearn.metrics import classification_report, confusion_matrix
import math
import random
import warnings
from collections import Counter
# Custom script 
#%matplotlibe inline

In [29]:
from load_data import load_dataset, batch_data
from utilities import *
if split == 'random':
    clients_batched, clients_test_data_batched, test_batched, p_Group, np_Group, sa_index, Xtr, features, sens_feature_list = load_dataset(dataset_name, split, num_clients)
else:
    clients_batched, client_data_testx, client_data_testy, test_batched, p_Group, np_Group, sa_index, Xtr, features, sens_feature_list = load_dataset(dataset_name, split, num_clients)

In [30]:
client_names = list(clients_batched.keys())
bs = list(clients_batched["client_1"])[0][0].shape[0] 
#first calculate the total training data points across clinets
global_count = sum([tf.data.experimental.cardinality(clients_batched[client_name]).numpy() for client_name in client_names])*bs
local_count = tf.data.experimental.cardinality(clients_batched['client_1']).numpy()*bs
print(global_count) 
print(local_count)

72000
24000


In [31]:
#print('Number of client datasets: {l}'.format(l=len(clients_batched)))
#print('First dataset: {d}'.format(d=clients_batched['client_1']))
#print('Number of client test datasets: {l}'.format(l=len(clients_test_data_batched)))

In [32]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,BatchNormalization,InputLayer
from tensorflow.keras.callbacks import EarlyStopping
class SimpleMLP:
    @staticmethod
    def build(x_train,n):
        
        model = Sequential()
        model.add(InputLayer(input_shape=(x_train.shape[1],)))
        model.add(Dense(x_train.shape[1],activation='relu'))#,input_shape=(x_train.shape[1],)))
        #model.add(BatchNormalization())
        model.add(Dropout(0.25))
        model.add(Dense(n*x_train.shape[1],activation='relu'))
        #model.add(BatchNormalization())
        model.add(Dropout(0.5))
        model.add(Dense(n*x_train.shape[1],activation='relu'))

        model.add(Dense(1,activation='sigmoid'))
        return model

In [33]:
lr = 0.00001 #0.0001, 0.00001, 0.0001, a_age = 0.0001, b_age = 0.0001, d_age = 0.0001 
comms_round = 50
loss='binary_crossentropy'
metrics = ['accuracy']
#optimizer = Adam(learning_rate=lr)
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=lr)


In [ ]:
from sklearn.utils import class_weight
smlp_global = SimpleMLP()
num_layers_mult=1
n=num_layers_mult
comms_round = 50
global_model = smlp_global.build(Xtr,n)
sensitivity_= []
specificity_= []
BalanceACC_= []
G_mean_= []
FP_rate_= []
FN_rate_= []
assigned_positives = 0
total_positives = 0
accuracy_= []
loss_= []
statistical_parity_ = []
eqop_ = []
ate_ = []
acc_ = []
disc_thresh = 0.005
lambda_initial = 0.05 #0.005,0.005, 0.008, a_age = 0.05
disc_tolerance = 0.25 #0.005,0.1
epsilon=1
pp_group = p_Group
npp_group=np_Group
#commence global training loop
import time
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
destination = "./"
start = time.time()
end_time = []
e=EarlyStopping(patience=5,restore_best_weights=True)
from imblearn.over_sampling import SMOTE

for comm_round in range(comms_round):
            
    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()
    
    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(clients_batched.keys())
    #random.shuffle(client_names)
    
    #loop through each client and create new local model
    client_number=0
    for client in client_names:   
        smlp_local = SimpleMLP()
        local_model = smlp_local.build(Xtr, n)
        local_model.compile(loss=loss, 
                      optimizer=optimizer, 
                      metrics=metrics)
        
        
        
        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)
        
        #here we should get the traning data from clients and try to do ksmote and see changes
       
        xxte = [] ##?client data?
        yte = []
        
        for(X_test, Y_test) in clients_batched[client]:
            
            i = 0
            while (i <len(X_test)):       
                xxte.append(X_test[i].numpy())  
                yte.append(Y_test[i].numpy())
                i +=1
        print(client)
        #print("length of dataset: %s" %len(xxte))
        if split=='random':
            x_test_client = []
            y_test_client = []
            for(X_test_real, Y_test_real) in clients_test_data_batched[client]:
                i = 0
                while (i <len(X_test_real)):       
                    x_test_client.append(X_test_real[i].numpy())  
                    y_test_client.append(Y_test_real[i].numpy())
                    i +=1
        
            x_test_client = np.array(x_test_client)
        
            y_test_client = np.array(y_test_client)
        else:
            y_test_client = client_data_testy[client_number]
            x_test_client = client_data_testx[client_number]
            #test_batched = tf.data.Dataset.from_tensor_slices((x_test_client, y_test_client)).batch(len(y_test_client))
       
        client_number+=1
        
        
   
        minority_label = 1
        majority_label = 0
        
        print("bismillah")
        if comm_round == 0:
            Xtr_1_new=np.array(xxte)
            Ytr_1_new=np.array(yte)
              
                    
            Xtr_1_new = np.array(Xtr_1_new)
            Ytr_1_new = np.array(Ytr_1_new)
            added_points = len(Xtr_1_new) - len(xxte)
            
            #new batch for new data
            #split data for validation set
            Xtr_1_new_split,x_val,Ytr_1_new_split,y_val=train_test_split(Xtr_1_new,Ytr_1_new,test_size=0.10,shuffle=True)
        
            data = list(zip(Xtr_1_new_split, Ytr_1_new_split))
            random.shuffle(data)
            btach_data = batch_data(data, bs=30)  
            
            class_weights=find_class_Weight(Ytr_1_new, majority_label,minority_label)
            local_model.fit(btach_data,validation_data=(x_val,y_val),validation_steps=1, callbacks=[e], epochs=3, verbose=0, 
                            class_weight=class_weights)
            
            if comm_round==0:
                local_model.save_weights('./checkpoints/my_checkpoint')
            pp_group = p_Group
            npp_group=np_Group
            
            sensitive_feature = sens_feature_list[0]
            x_test_client_df = pd.DataFrame(x_test_client, columns=features)
            s_client = x_test_client_df[sensitive_feature]
            s_client_list = s_client.tolist()
            ytest_potential = find_potential_outcomes(x_test_client_df,y_test_client, sensitive_feature)
            
            disc_score, assigned_positives, total_positives, balanced_accuracy_client = test_client_model(x_test_client, y_test_client,  local_model,pp_group,npp_group, sa_index, fairness_notion, ytest_potential, s_client_list, sensitive_feature)
            print("balanced_accuracy_client %s" % balanced_accuracy_client)
            
        
            '''
            if fairness_notion == 'ate':
                #sensitive_feature = "sex"
                sensitive_feature = sens_feature_list[0]
                x_test_client_df = pd.DataFrame(x_test_client, columns=features)
                y_test_client_df = pd.DataFrame(y_test_client, columns=['label'])
                s_client = x_test_client_df[sensitive_feature]
                ytest_potential = find_potential_outcomes(x_test_client_df,y_test_client, sensitive_feature)
                y_test_client_tensor = torch.tensor(y_test_client_df.values, dtype=torch.float32)
                s_client_list = s_client.tolist()
                disc_score, assigned_positives, total_positives, balanced_accuracy_client = test_client_model(x_test_client, y_test_client,  local_model,pp_group,npp_group, sa_index, fairness_notion, s_client_list, y_test_client_tensor, ytest_potential)
            '''
            if disc_score<0:
                pp_group = np_Group
                npp_group=p_Group
                disc_score, assigned_positives, total_positives, balanced_accuracy_client = test_client_model(x_test_client, y_test_client,  local_model,pp_group,npp_group,  sa_index, fairness_notion, ytest_potential, s_client_list, sensitive_feature)
            else:
                pp_group = p_Group
                npp_group=np_Group
            trade_off = (1+epsilon**2)*((balanced_accuracy_client*(1-abs(disc_score)))/(epsilon*balanced_accuracy_client+(1-abs(disc_score))))
            lambda_score = lambda_initial*(1+(disc_score/disc_tolerance))
            
        
        if comm_round!=0:
            pp_group = p_Group
            npp_group=np_Group
            disc_score, assigned_positives, total_positives, balanced_accuracy_client = test_client_model(x_test_client, y_test_client,  local_model,pp_group,npp_group,  sa_index, fairness_notion, ytest_potential, s_client_list, sensitive_feature)
            
            if disc_score<0:
                pp_group = np_Group
                npp_group=p_Group
                disc_score, assigned_positives, total_positives, balanced_accuracy_client = test_client_model(x_test_client, y_test_client,  local_model,pp_group,npp_group,  sa_index, fairness_notion, ytest_potential, s_client_list, sensitive_feature)
            else:
                pp_group = p_Group
                npp_group=np_Group
            trade_off = (1+epsilon**2)*((balanced_accuracy_client*(1-abs(disc_score)))/(epsilon*balanced_accuracy_client+(1-abs(disc_score))))
        
        if disc_score > disc_thresh:
            
      
                prev_xtr,prev_ytr=[],[]
                prev_disc_score= []
                prev_trade_off=[]
                lambda_score = lambda_initial*(1+(disc_score/disc_tolerance))
                greater_disc_score = 0
                min_disc_score =disc_score
                max_trade_off = trade_off
                Xtr_1_new=np.array(xxte)
                Ytr_1_new=np.array(yte)
                prev_xtr,prev_ytr = Xtr_1_new, Ytr_1_new
                class_weights=find_class_Weight(Ytr_1_new, majority_label,minority_label)
                points = len(Xtr_1_new) - len(xxte)
                itr = 0
                
                while disc_score > disc_thresh:
                    itr+=1
                    closest_to_zero = min(disc_score, min_disc_score, key=abs)
                    if trade_off>max_trade_off:
                    #if closest_to_zero != min_disc_score:
                        min_disc_score = disc_score
                        max_trade_off = trade_off
                        prev_xtr,prev_ytr = Xtr_1_new, Ytr_1_new
                        points = len(Xtr_1_new) - len(xxte)
                        local_model.save_weights('./checkpoints/my_checkpoint')
                    if assigned_positives<= total_positives:
                        
                        #dmin_p_x, dmin_np_x, dmaj_p_x, dmaj_np_x = get_statistics(Xtr_1_new,Ytr_1_new,minority_label,majority_label)
                        #N(C+,S−) =N(C+,S−)+λ× N(C−,S−)
                        Xtr_min_p_new,Ytr_min_p_new = create_synth_data(xxte, yte, minority_label,majority_label,5,lambda_score,'min_p',pp_group,npp_group, sa_index)
                    
                        #N(C−,S−) =N(C−,S−)−λ× N(C−,S−)
                        Xtr_maj_p_new,Ytr_maj_p_new = downsample(xxte, yte, minority_label,majority_label,5,lambda_score,'maj_p',pp_group,npp_group, sa_index)    
                        
                        for k in Xtr_maj_p_new:
                            Xtr_min_p_new.append(k)
                        for k in Ytr_maj_p_new:
                            Ytr_min_p_new.append(k)
                        
                        Xtr_1_new = np.array(Xtr_min_p_new)
                        Ytr_1_new = np.array(Ytr_min_p_new)
                        #dmin_p_x2, dmin_np_x2, dmaj_p_x2, dmaj_np_x2 = get_statistics(Xtr_1_new,Ytr_1_new,minority_label,majority_label)
                         
                    else:
                        #dmin_p_x, dmin_np_x, dmaj_p_x, dmaj_np_x = get_statistics(Xtr_1_new,Ytr_1_new,minority_label,majority_label)
                        #N(C−,S+) =N(C−,S+)+λ× N(C+,S+)
                        Xtr_maj_np_new,Ytr_maj_np_new = create_synth_data(xxte, yte, minority_label,majority_label,5,lambda_score,'maj_np',pp_group,npp_group, sa_index)
                    
                        #N(C+,S+) =N(C+,S+)−λ× N(C+,S+)
                        Xtr_min_np_new,Ytr_min_np_new = downsample(xxte, yte, minority_label,majority_label,5,lambda_score,'min_np',pp_group,npp_group, sa_index)
                        
                        for k in Xtr_min_np_new:
                            Xtr_maj_np_new.append(k)
                        for k in Ytr_min_np_new:
                            Ytr_maj_np_new.append(k)
                        
                        Xtr_1_new = np.array(Xtr_maj_np_new)
                        Ytr_1_new = np.array(Ytr_maj_np_new)
                       
                    added_points = len(Xtr_1_new) - len(xxte)
                    #new batch for new data
                    #split data for validation set
                    Xtr_1_new_split,x_val,Ytr_1_new_split,y_val=train_test_split(Xtr_1_new,Ytr_1_new,test_size=0.10,shuffle=True)
                    
                    data = list(zip(Xtr_1_new_split, Ytr_1_new_split))
                    random.shuffle(data)
                    btach_data = batch_data(data, bs=30)  
                    #fit local model with client's data
                    #create validation data and early stop
                    
                    local_model.fit(btach_data,validation_data=(x_val,y_val),validation_steps=1, callbacks=[e], epochs=3, verbose=0,
                                    class_weight = class_weights)
                    
                    disc_score, assigned_positives, total_positives, balanced_accuracy_client = test_client_model(x_test_client, y_test_client,  local_model,pp_group,npp_group,  sa_index, fairness_notion, ytest_potential, s_client_list, sensitive_feature)
                    
                    trade_off = (1+epsilon**2)*((balanced_accuracy_client*(1-abs(disc_score)))/(epsilon*balanced_accuracy_client+(1-abs(disc_score))))
                    
                    closest_to_zero = min(disc_score, min_disc_score, key=abs)
                    if trade_off>max_trade_off:
                    
                        
                        min_disc_score = disc_score
                        max_trade_off = trade_off
                        prev_xtr,prev_ytr = Xtr_1_new, Ytr_1_new
                        points = len(Xtr_1_new) - len(xxte)
                        local_model.save_weights('./checkpoints/my_checkpoint')
               
                    if len(prev_trade_off)>0:
                       
                        if trade_off<prev_trade_off[-1]:
                            greater_disc_score+=1
                            print(greater_disc_score)
                            if greater_disc_score >2:
                                local_model.load_weights('./checkpoints/my_checkpoint')
                                added_points = points
                                break
                     
                
                    prev_trade_off.append(trade_off)
                    xxte, yte = Xtr_1_new, Ytr_1_new
                
                xxtee=[]
                ytee=[]
                dataset = tf.data.Dataset.from_tensor_slices((list(prev_xtr), list(prev_ytr)))
                up_dict = {client:batch_data(dataset)}
                clients_batched.update(up_dict)
                #clients_batched[client_name] = batch_data(dataset)
                for(X_test, Y_test) in clients_batched[client]:
            
                    i = 0
                    while (i <len(X_test)):       
                        xxtee.append(X_test[i].numpy())  
                        ytee.append(Y_test[i].numpy())
                        i +=1 
                
        else:
          
            if comm_round!=0:
                Xtr_1_new=np.array(xxte)
                Ytr_1_new=np.array(yte)
                #class_weights = class_weight.compute_class_weight('balanced',
                #                                 np.unique(Ytr_1_new),
                #                                 Ytr_1_new)
                #class_weights = {majority_label:1, minority_label:7}
                class_weights=find_class_Weight(Ytr_1_new, majority_label,minority_label)
                Xtr_1_new,x_val,Ytr_1_new,y_val=train_test_split(Xtr_1_new,Ytr_1_new,test_size=0.10,shuffle=True)
                data = list(zip(Xtr_1_new,Ytr_1_new))
                #random.shuffle(data)
                added_points = 0
                btach_data = batch_data(data, bs=30)
                
                local_model.fit(btach_data,validation_data=(x_val,y_val),validation_steps = 1, callbacks=[e], epochs=10, 
                                verbose=0, class_weight = class_weights)
        
        #scale the model weights and add to list       
        client_names = list(clients_batched.keys())
        bs = list(clients_batched[client])[0][0].shape[0]
        #first calculate the total training data points across clinets
        global_count = sum([tf.data.experimental.cardinality(clients_batched[client_name]).numpy() for client_name in client_names])*bs
        global_count = global_count + (added_points)
        # get the total number of data points held by a client
        local_count = len(Xtr_1_new)
        scaling_factor = local_count/global_count
                
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)
        
        #clear session to free memory after each communication round
        K.clear_session()
        
    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)
    #update global model 
    
    global_model.set_weights(average_weights)
    
    #test global model and print out metrics after each communications round
    for(X_test, Y_test) in test_batched:
        
        X_test_df = pd.DataFrame(X_test, columns=features)
        s_ = X_test_df[sensitive_feature]
        s_list = s_.tolist()
        Y_test_potential = find_potential_outcomes(X_test_df,Y_test, sensitive_feature)
        global_acc, global_loss,conf,stat_parity,eqop,ate = test_model(X_test,Y_test, p_Group, np_Group,  sa_index, global_model, comm_round, Y_test_potential, s_list, sensitive_feature)
        print("x_test %s "% len(X_test))
        TN = conf[0][0]
        FP = conf[0][1]
        FN = conf[1][0]
        TP = conf[1][1]
        sensitivity = TP/(TP+FN) 
        specificity = TN/(FP+TN)
        
        acc = (TP+TN)/(TP+TN+FP+FN)
        BalanceACC = (sensitivity+specificity)/2
        G_mean = math.sqrt(sensitivity*specificity)
        FN_rate= FN/(FN+TP) 
        FP_rate = FP/(FP+TN) 
        #add the data to arrays
        sensitivity_.append(sensitivity)
        specificity_.append(specificity)
        BalanceACC_.append(BalanceACC)
        G_mean_.append(G_mean)
        FP_rate_.append(FP_rate)
        FN_rate_.append(FN_rate)
        accuracy_.append(global_acc)
        loss_.append(global_loss)
        statistical_parity_.append(stat_parity)
        eqop_.append(eqop)
        ate_.append(ate)
        acc_.append(acc)
    print(acc)
    print(BalanceACC)
    print(ate)
    print(stat_parity)
